<a href="https://colab.research.google.com/github/chandru-master/twitter-tweets-counter/blob/main/count_on_the_top_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install findspark

In [3]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 53.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=cf727603d4569e38df3250729b5b85f1f8cc60ba6e20df34d55e1dd15b880fe3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [16]:
import findspark
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob
import sys
import requests
import socket
import pyspark.sql.functions as f

In [5]:
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("words").getOrCreate()

In [8]:
sc = spark.sparkContext

In [10]:
df = spark.read.csv('tweet.csv', header=True)

In [11]:
df.show()

+---+--------------------+--------+--------------------+
|_c0|                Date|    User|               Tweet|
+---+--------------------+--------+--------------------+
|  0|2019-12-31 21:37:...|elonmusk|@engineers_feed @...|
|  1|2019-12-31 06:59:...|elonmusk|@JohnnaCrider1 It...|
|  2|2019-12-31 06:57:...|elonmusk|@newscientist Exp...|
|  3|2019-12-31 02:27:...|elonmusk|@teslaownersSV @r...|
|  4|2019-12-30 23:27:...|elonmusk|Rest in peace Syd...|
|  5|2019-12-30 23:09:...|elonmusk|@kulpability @cle...|
|  6|2019-12-30 23:00:...|elonmusk|@John_Gardi @Erda...|
|  7|2019-12-30 22:44:...|elonmusk|@ShaneAppleton7 @...|
|  8|2019-12-30 22:41:...|elonmusk|@JaneidyEve Heade...|
|  9|2019-12-30 22:39:...|elonmusk|@EvaFoxU Biting o...|
| 10|2019-12-30 10:05:...|elonmusk|@Teslarati Hard t...|
| 11|2019-12-30 10:02:...|elonmusk|@BaconMan65 @Phil...|
| 12|2019-12-30 10:00:...|elonmusk|@cleantechnica Wi...|
| 13|2019-12-30 09:56:...|elonmusk|@EverydayTesla Ye...|
| 14|2019-12-30 09:18:...|elonm

In [12]:
df.head()

Row(_c0='0', Date='2019-12-31 21:37:06+00:00', User='elonmusk', Tweet='@engineers_feed @physicsJ It’s a bit slow')

In [13]:
df.createOrReplaceTempView('filter_view')

In [14]:
df_filter = spark.sql("""select Tweet from filter_view""")


In [15]:
df_filter.show()

+--------------------+
|               Tweet|
+--------------------+
|@engineers_feed @...|
|@JohnnaCrider1 It...|
|@newscientist Exp...|
|@teslaownersSV @r...|
|Rest in peace Syd...|
|@kulpability @cle...|
|@John_Gardi @Erda...|
|@ShaneAppleton7 @...|
|@JaneidyEve Heade...|
|@EvaFoxU Biting o...|
|@Teslarati Hard t...|
|@BaconMan65 @Phil...|
|@cleantechnica Wi...|
|@EverydayTesla Ye...|
|Barrel on dome ht...|
|@ajtourville @cle...|
|@SteveHamel16 @cl...|
|@DonaldM38768041 ...|
|@cleantechnica Se...|
| @Erdayastronaut Yes|
+--------------------+
only showing top 20 rows



count the total number of words for each colums string values of  tweets

In [19]:
df = df_filter.withColumn('wordCount', f.size(f.split(f.col('Tweet'), ' ')))
df.show()

+--------------------+---------+
|               Tweet|wordCount|
+--------------------+---------+
|@engineers_feed @...|        6|
|@JohnnaCrider1 It...|        5|
|@newscientist Exp...|        4|
|@teslaownersSV @r...|       18|
|Rest in peace Syd...|        9|
|@kulpability @cle...|        4|
|@John_Gardi @Erda...|       34|
|@ShaneAppleton7 @...|       11|
|@JaneidyEve Heade...|       12|
|@EvaFoxU Biting o...|       13|
|@Teslarati Hard t...|       17|
|@BaconMan65 @Phil...|        3|
|@cleantechnica Wi...|        8|
|@EverydayTesla Ye...|       15|
|Barrel on dome ht...|        4|
|@ajtourville @cle...|       11|
|@SteveHamel16 @cl...|        6|
|@DonaldM38768041 ...|        9|
|@cleantechnica Se...|        5|
| @Erdayastronaut Yes|        2|
+--------------------+---------+
only showing top 20 rows



# Need to count same repeated words and which is the most 
# if we have the socket connection for the streamed data we can use trigger funtion for 300 seconds lyk.trigger(300 = seconds) and we can calclulate since we are using sample data set which is extracted from snscrape we will use with column methods instead of flattening the data we can count with splits

In [21]:
df.withColumn('word', f.explode(f.split(f.col('Tweet'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show()

+---------------+-----+
|           word|count|
+---------------+-----+
|             to| 1160|
|             is|  817|
|              a|  778|
|            the|  761|
|             of|  727|
|          &amp;|  700|
|             in|  597|
|            for|  530|
|         @Tesla|  418|
|             be|  404|
|             on|  386|
|          Tesla|  376|
|           will|  353|
|            but|  336|
|@Erdayastronaut|  311|
|           with|  289|
|           that|  264|
|             it|  252|
|              I|  227|
|            are|  220|
+---------------+-----+
only showing top 20 rows



In [24]:
df_filter.show(truncate=0)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|@engineers_feed @physicsJ It’s a bit slow                                                                                                                                                                                          |
|@JohnnaCrider1 It’s not ready yet                                              

In [25]:
df.write.parquet("count.parquet")

# now successfully saved as parquet file